In [1]:
import pandas as pd
import numpy as np

In [2]:
matches = pd.read_csv('/Users/tliu/Desktop/Erdos Project/Data/matches.csv')

In [3]:
matches.head()

,player1,player2,score1,score2,best_of,tournament_id,date,year
0,Colin Roscoe,Jackie Rea,9,6,17,753,NaN,1982
1,Tommy Murphy,Clive Everton,9,4,17,753,NaN,1982
2,Vic Harris,Marcus Owen,9,4,17,753,NaN,1982
3,Bob Harris,Graham Cripsey,9,6,17,753,NaN,1982
4,Geoff Foulds,Matt Gibson,9,3,17,753,NaN,1982


In [4]:
tournaments = list(matches.tournament_id.unique())


In [5]:
matches[matches['tournament_id'] == tournaments[1040]]

,player1,player2,score1,score2,best_of,tournament_id,date,year
111102,Xing Zihao,Fergal O'Brien,4,3,7,5848,2023-11-02,2023
111103,Cao Yupeng,David Lilley,4,2,7,5848,2023-10-30,2023
111104,Zhou Yuelong,Anthony Hamilton,4,0,7,5848,2023-10-30,2023
111105,Kyren Wilson,Mostafa Dorgham,4,2,7,5848,2023-12-11,2023
111106,Gary Wilson,Elliot Slessor,4,3,7,5848,2023-12-11,2023
...,...,...,...,...,...,...,...,...
111222,Noppon Saengkham,Sanderson Lam,5,3,9,5848,2023-12-15,2023
111223,John Higgins,Tom Ford,5,3,9,5848,2023-12-15,2023
111224,Gary Wilson,Zhou Yuelong,6,5,11,5848,2023-12-16,2023
111225,Noppon Saengkham,John Higgins,6,3,11,5848,2023-12-16,2023


In [6]:
#Using matches history of before 202x to predict matches in 202x
from elo_v3 import elo_rating
from sklearn.metrics import accuracy_score
Ks = [6]
nth = [1080]
scores = np.zeros((5,4))


i=0
for K in Ks:
    j=0
    for n in nth:
        past_tournaments = tournaments[:n]
        current = tournaments[n]
        past_index = matches['tournament_id'].map(lambda x: x in past_tournaments)
        

        matches_past = matches[past_index]
        matches_future = matches[matches['tournament_id'] == current]

        #Update statistics of players
        elo = elo_rating()
        elo.update_stats(matches = matches_past, K_factor = K, ratio = 0.1)

        #Store the actual result in y
        matches_future['result'] = 0  
        y = np.array(matches_future['result'])

        #Making prediction
        X = matches_future[['player1', 'player2', 'best_of']]
        prediction, _ = elo.predict(X)

        #Record the accuracy score
        from sklearn.metrics import accuracy_score
        scores[i,j] = accuracy_score(prediction, y)
        
        j += 1
    i+=1




/Users/tliu/Desktop/Erdos Project/Elo Rating/elo_v3/elo_v3.py:120: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.stats_by_years.fillna(0, inplace = True)
/var/folders/pd/b57yb11s57b55h2t3r3c8q1r0000gn/T/ipykernel_1567/533775058.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches_future['result'] = 0


In [7]:
print(scores)

[[0.73239437 0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]]


Ks = [5,6,7,8]
years = [2024, 2025]

[[0.64574616 0.62      ]
 [0.64888424 0.62      ]
 [0.64818689 0.6224    ]
 [0.64888424 0.624     ]]

Ks = [5,6,7,8]
years = [2020, 2021, 2022, 2023]

[[0.6726283  0.65418652 0.65121333 0.63888889]
 [0.67379471 0.65384615 0.65157552 0.64308786]
 [0.67534992 0.65588836 0.65012677 0.64696382]
 [0.67068429 0.65724983 0.64831583 0.64599483]]

Ks = [3,6,8,10]
nth = [1000, 1050, 1070, 1080]

[[0.68695652 0.66666667 0.59055118 0.69014085]
 [0.67826087 0.66666667 0.59055118 0.73239437]
 [0.67826087 0.6        0.59055118 0.71830986]
 [0.64347826 0.6        0.5984252  0.73943662]]

Ks = [12, 15, 18]
nth = [1000, 1050, 1070, 1080]

[[0.65217391 0.6        0.60629921 0.72535211]
 [0.65217391 0.6        0.60629921 0.73239437]
 [0.64347826 0.66666667 0.59055118 0.73239437]
 [0.         0.         0.         0.        ]]

Ks = [20,25,30]
nth = [1000, 1050, 1070, 1080]

[[0.63478261 0.66666667 0.58267717 0.73239437]
 [0.63478261 0.6        0.59055118 0.71126761]
 [0.63478261 0.6        0.58267717 0.71126761]
 [0.         0.         0.         0.        ]]

Ks = [4, 6, 8, 15, 20]
nth = [750, 999, 1040, 1080]

[[0.72222222 0.63779528 0.672      0.71126761]
 [0.72222222 0.65354331 0.688      0.73239437]
 [0.73809524 0.65354331 0.696      0.71830986]
 [0.70634921 0.63779528 0.696      0.73239437]
 [0.70634921 0.63779528 0.728      0.73239437]]